In [1]:
!pip install torch transformers flask beautifulsoup4 requests nltk


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install accelerate -U


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from transformers import GPT2TokenizerFast, GPTNeoForCausalLM, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
import torch

In [ ]:
# Tokenizer and model setup
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-2.7B')

In [ ]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, inputs, outputs, max_length=512):
        self.tokenizer = tokenizer
        self.inputs = inputs
        self.outputs = outputs
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]
        
        input_encodings = tokenizer(input_text, padding="max_length", truncation=True, max_length=self.max_length)
        output_encodings = tokenizer(output_text, padding="max_length", truncation=True, max_length=self.max_length)
        
        return {
            "input_ids": torch.tensor(input_encodings['input_ids']),
            "attention_mask": torch.tensor(input_encodings['attention_mask']),
            "labels": torch.tensor(output_encodings['input_ids'])
        }
    

# Load and preprocess data
df = pd.read_json('Psychology-10K.json')
inputs = df['input'].tolist()
outputs = df['output'].tolist()
dataset = TextDataset(tokenizer, inputs, outputs)

In [14]:

# Training arguments and setup
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    logging_steps=10,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)


model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [12]:
# Train the model
trainer.train()

  0%|          | 0/1970 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
def predict_response(input_text, model, tokenizer):
    # Preprocess and tokenize the input text
    encoded_input = tokenizer.encode_plus(
        input_text,
        max_length=512,  # Ensure the length matches what was used during training
        truncation=True,
        padding='max_length',
        return_tensors='pt'  # Return PyTorch tensors
    )

    # Move the tensors to the same device as the model
    encoded_input = {key: tensor.to(model.device) for key, tensor in encoded_input.items()}

    # Make prediction
    with torch.no_grad():
        output = model(**encoded_input)

    # Retrieve the probabilities (logits) and find the predicted class
    logits = output.logits
    predicted_class_index = logits.argmax(-1).item()

    # If you used LabelEncoder during preprocessing, you need to reverse that encoding:
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_label

# Example usage
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Make sure the model is on the right device
input_text = "i feel lonely"
response = predict_response(input_text, model, tokenizer)
print("Predicted Response:", response)

NameError: name 'model' is not defined